# Chat UDP

En esta sesión de laboratorio vamos a realizar un ejemplo mínimo de programación con sockets en Python3. Todo (o casi todo) lo que vamos a realizar en esta sesión se podría realizar utilizando herramientas de terminal como `ncat`, pero el objetivo es dar unos primeros pasos con Python 3 y la utilización de la librería `socket` para realizar conexiones de red.

## ¿Por qué UDP?

Como ya os sonará de la asignatura Redes 1, en la **capa de transporte** de la pila TCP/IP existen 2 protocolos principales: _Transport Control Protocol_, o TCP, y _User Datagram Protocol_, o UDP.

La principal similitud entre ambos protocolos es que ambos utilizan un número natural como direccionamiento: el puerto. En ambos protocolos el valor del puerto puede ir desde 0 a 65536.

En cuanto a difrencias, el transporte realizado mediante TCP está orientado a conexión: existen dentro del protocolo una serie de herramientas para comprobar que los mensajes que se envían son recibidos por el destinatario, y herramientas para que si no se tiene esa confirmación, se reenvíe el paquete.

En cuanto a UDP, **no está orientado a conexión**, no dispone de herramientas para comprobar que un mensaje ha sido recibido y entregado al destinatario, ni siquiera sabemos si el destinatario está o no escuchando en su puerto.

En esta sesión vamos a utilizar UDP por su simplicidad para una primera aproximación.

## Hello world

En éste primer ejemplo vamos a escribir dos programas que se comuniquen a través de UDP con las siguientes características:

- Un cliente que envíe un mensaje que contenga la palabra "hello".
- Un servidor que imprima el mensaje recibido por la pantalla en el terminal.

### El cliente

In [1]:
#!/usr/bin/env python3

from socket import socket, AF_INET, SOCK_DGRAM  # Importamos del módulo socket la clase socket y las constantes para sockets IP y datagramas

sock = socket(AF_INET, SOCK_DGRAM)  # Creamos el socket de familia IP, tipo UDP (datagrama)
destination_address = ("localhost", 12345)  # Definimos la direccion de destino con una tupla de IP y puerto


sock.sendto(b"hello", destination_address)  # Enviamos la secuencia de bytes representada por "hello" a la dirección indicada anteriormente
sock.close()

Para poder ejecutar el ejemplo anterior, copiarlo y pegarlo en un editor de texto, guardar el fichero y dadle permisos. También podéis descargarlo desde [1](src/udp-chat/client1.py).

```bash
chmod u+x client1.py
./client.py
```

### El servidor

In [ ]:
#!/usr/bin/env python3

from socket import socket, AF_INET, SOCK_DGRAM

sock = socket(AF_INET, SOCK_DGRAM)
sock.bind(("", 12345))  # Con esta línea le decimos al socket que queremos usar el puerto 12345 para escuchar mensajes

message, client = sock.recvfrom(1024)  # El método devolvera una tupla con el mensaje recibido y la dirección del cliente. 1024 es el tamaño del buffer de recepción.

print(f"{message.decode()} from {client}")

sock.close()

Al igual que el ejemplo anterior, puede descargarse desde [2](src/udp-chat/server1.py) o copiar y pegar el código en un fichero nuevo y darle permisos de la misma manera:

```bash
chmod u+x server1.py
./server.py
```

## Hello world con respuesta

A continuación vamos a modificar el código anterior para que el servidor envíe una respuesta al cliente. Además, el cliente mostrará dicha respuesta por su salida estándar:

In [ ]:
#!/usr/bin/env python3

from socket import socket, AF_INET, SOCK_DGRAM

sock = socket(AF_INET, SOCK_DGRAM)

sock.sendto("Hello".encode(), ("localhost", 12345))

msg, server = sock.recvfrom(1024)
print(msg.decode(), " from ", server)

sock.close()

In [ ]:
#!/usr/bin/env python3

from socket import socket, AF_INET, SOCK_DGRAM

sock = socket(AF_INET, SOCK_DGRAM)
sock.bind(("", 12345))

msg, client = sock.recvfrom(1024)
print(msg.decode(), " from ", client)

sock.sendto("hi there".encode(), client)

sock.close()

## Entrada de usuario

Ahora vamos a escribir una pequeña aplicación de chat cliente-servidor. Deberá cumplir con lo siguiente:

- Comunicación bidireccional, pero no simultánea (el cliente inicia la conversación).
- La aplicación termina si cualquiera de los dos envía la cadena "bye".

### Pistas
- Para leer una cadena desde la entrada estándar desde un programa python usaremos la función `input()`.
- Para terminar un bucle podemos utilizar `break`.

## Simultáneos

Modificaremos el ejercicio anterior de manera que cualquiera de los pares sea capaz de enviar y recibir en cualquier momento (no por turnos).

### Pistas

- Usaremos hilos